## Import all required libraries and install tranformers

In [ ]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
import spacy 
from spacy import displacy
#import neptune.new as neptune                          
from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset, Dataset)

In [ ]:
!pip install emoji -q
!pip install datasets -q
!pip install rouge_score -q


!pip install transformers -q
!pip install sentencepiece -q
!pip install pandarallel -q

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

from transformers import AdamW, get_linear_schedule_with_warmup,PegasusConfig, PegasusTokenizer, PegasusForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, set_seed, TrainingArguments, Trainer, default_data_collator, AdamW, get_linear_schedule_with_warmup, Seq2SeqTrainer, Seq2SeqTrainingArguments


     |████████████████████████████████| 170 kB 6.8 MB/s 
     |████████████████████████████████| 298 kB 5.0 MB/s 
     |████████████████████████████████| 243 kB 59.2 MB/s 
     |████████████████████████████████| 1.1 MB 56.1 MB/s 
     |████████████████████████████████| 61 kB 622 kB/s 
     |████████████████████████████████| 132 kB 67.8 MB/s 
     |████████████████████████████████| 271 kB 67.1 MB/s 
     |████████████████████████████████| 160 kB 66.6 MB/s 
     |████████████████████████████████| 192 kB 69.5 MB/s 
     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 58.3 MB/s 
     |████████████████████████████████| 596 kB 63.5 MB/s 
     |████████████████████████████████| 3.3 MB 56.9 MB/s 
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


## Finetune Pegasus on ArgKP Dataset

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = PegasusConfig.from_pretrained(model_name)
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [ ]:
src_text = [""" PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""]
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

["California's largest electricity provider has turned off power to hundreds of thousands of customers."]


In [ ]:
src_text = ["""This is a great little neighborhood pet food market that also has grooming available. More on that later. There are a lot of an donation that i have practiced yoga on the extra fries and see some of just another rice. It took me to my side and really a free birthday it."""]
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

["It's my birthday today and to celebrate i went to the pet food market."]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# dataset_path = '/content/drive/MyDrive/train_dev_test_dataset/'
# gold_data_dir = '/content/drive/MyDrive/train_dev_test_dataset/'

Mounted at /content/drive


In [ ]:
# df_train = pd.read_csv(dataset_path + 'arguments_train.csv')
# df_train.head()

In [ ]:
# df_val = pd.read_csv(dataset_path + 'val_tfidf.csv')
# df_test = pd.read_csv(dataset_path + 'final_test.csv')

In [ ]:
def load_kpm_data(gold_data_dir, subset):
    arguments_file = os.path.join(gold_data_dir, f"arguments_{subset}.csv")
    key_points_file = os.path.join(gold_data_dir, f"key_points_{subset}.csv")
    labels_file = os.path.join(gold_data_dir, f"labels_{subset}.csv")

    arguments_df = pd.read_csv(arguments_file)
    key_points_df = pd.read_csv(key_points_file)
    labels_file_df = pd.read_csv(labels_file)

    return arguments_df, key_points_df,labels_file_df

In [ ]:
def give_tuples(df_labels,df_arguments,df_key_points):

  arguments = []
  key_points = []
  topics = []
  stance = []

  for i,j in zip(df_labels['arg_id'],df_labels['key_point_id']):

    
    idx_arg = df_arguments[df_arguments['arg_id'] == i ].argument.index[0]
    idx_kp = df_key_points[df_key_points['key_point_id'] == j ].key_point.index[0]

    arguments.append(df_arguments[df_arguments['arg_id'] == i]['argument'][idx_arg])
    topics.append(df_arguments[df_arguments['arg_id'] == i]['topic'][idx_arg])
    key_points.append(df_key_points[df_key_points['key_point_id'] == j]['key_point'][idx_kp])
    stance.append(df_key_points[df_key_points['key_point_id'] == j]['stance'][idx_kp])

  return arguments,key_points,topics, stance


In [ ]:
df_arguments,df_key_points,df_labels = load_kpm_data(gold_data_dir,'train')
argument_train, key_point_train, topic_train,stance_train   = give_tuples(df_labels,df_arguments,df_key_points)
df_train = pd.DataFrame([], columns = ['arg_topic', 'key_point'])
df_train['arg_topic'] = argument_train
df_train['key_point'] = key_point_train
print(df_train.shape) 

In [ ]:
df_arguments,df_key_points,df_labels = load_kpm_data(gold_data_dir,'dev')
argument_train, key_point_train, topic_train,stance_train   = give_tuples(df_labels,df_arguments,df_key_points)
df_val = pd.DataFrame([], columns = ['arg_topic', 'key_point'])
df_val['arg_topic'] = argument_train
df_val['key_point'] = key_point_train
print(df_val.shape)

In [ ]:
df_arguments,df_key_points,df_labels = load_kpm_data(gold_data_dir,'test')
argument_train, key_point_train, topic_train,stance_train   = give_tuples(df_labels,df_arguments,df_key_points)
df_test = pd.DataFrame([], columns = ['arg_topic', 'key_point'])
df_test['arg_topic'] = argument_train
df_test['key_point'] = key_point_train
print(df_test.shape)

In [ ]:
df_train = pd.concat([df_train,df_val])

In [ ]:
dataset_path = '/content/drive/MyDrive/gen_datasets/'
df_train.to_csv('/content/drive/MyDrive/gen_datasets/' + 'gen_train.csv')
df_val.to_csv('/content/drive/MyDrive/gen_datasets/' + 'gen_val.csv')
df_test.to_csv('/content/drive/MyDrive/gen_datasets/' + 'gen_test.csv')

In [ ]:
dataset_path = '/content/drive/MyDrive/gen_datasets/'
df_train = pd.read_csv(dataset_path + 'gen_train.csv')
df_val = pd.read_csv(dataset_path + 'gen_val.csv')
df_test = pd.read_csv(dataset_path + 'gen_test.csv')

In [ ]:
import datasets

# Get the column names for input/target.
dataset_columns = ('arg_topic', 'key_point')
arg_topic_column = dataset_columns[0]
key_point_column = dataset_columns[1]

# Temporarily set max_target_length for training.
max_source_length = 256
max_target_length = 64
padding = "max_length" 

device='cuda'

train_dataset = datasets.load_dataset('csv', data_files = dataset_path + 'gen_train.csv')['train']
val_dataset = datasets.load_dataset('csv', data_files = dataset_path + 'gen_val.csv')['train']
test_dataset = datasets.load_dataset('csv', data_files = dataset_path + 'gen_test.csv')['train']


Using custom data configuration default-f8f880f0dddceb52


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f8f880f0dddceb52/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

Using custom data configuration default-2a97231bf073579c


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2a97231bf073579c/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data = {'train': train_dataset, 'validation': val_dataset, 'test': test_dataset}
column_names = data["train"].column_names
prefix=""

In [ ]:
def preprocess_function(examples):
    inputs = examples[arg_topic_column]
    targets = examples[key_point_column]
    inputs = [prefix + inp for inp in inputs]
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=column_names,
            load_from_cache_file=False,
        )

eval_dataset = val_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=column_names,
            load_from_cache_file=False,
        )

test_dataset = test_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=column_names,
            load_from_cache_file=False,
        )


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
label_pad_token_id = -100 
data_collator = default_data_collator

metric_name = "rouge"
metric = datasets.load_metric(metric_name)


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    if metric_name == "rouge":
        result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        # Extract a few results from ROUGE
        result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    else:
        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


In [ ]:
def gen_arg_topic(df):
    arg_topic = []
    for i in range(len(df)):
        arg_topic.append(df['argument'][i] + df['topic'][i])
    df['arg_topic'] = arg_topic
    return df


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/Pegasus-ArgKP',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/drive/MyDrive/Pegasus-ArgKP',            # directory for storing logs
    save_steps = 4518, #in order to store the last model at the end of 3 epochs
    predict_with_generate=True
)

# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [ ]:
torch.cuda.empty_cache()
model.cuda();

In [ ]:
checkpoint = ''
train_result = trainer.train(resume_from_checkpoint=None)
trainer.save_model()

***** Running training *****
  Num examples = 24093
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9036


Step,Training Loss
500,3.114000
1000,1.044200
1500,0.514500
2000,0.395600
2500,0.333100


Step,Training Loss
500,3.114000
1000,1.044200
1500,0.514500
2000,0.395600
2500,0.333100
3000,0.302400
3500,0.280000
4000,0.267200
4500,0.253300
5000,0.238700


Saving model checkpoint to /content/drive/MyDrive/Pegasus-ArgKP/checkpoint-4518
Configuration saved in /content/drive/MyDrive/Pegasus-ArgKP/checkpoint-4518/config.json
Model weights saved in /content/drive/MyDrive/Pegasus-ArgKP/checkpoint-4518/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Pegasus-ArgKP/checkpoint-4518/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Pegasus-ArgKP/checkpoint-4518/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/Pegasus-ArgKP/checkpoint-9036
Configuration saved in /content/drive/MyDrive/Pegasus-ArgKP/checkpoint-9036/config.json
Model weights saved in /content/drive/MyDrive/Pegasus-ArgKP/checkpoint-9036/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Pegasus-ArgKP/checkpoint-9036/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Pegasus-ArgKP/checkpoint-9036/special_tokens_map.json


Training completed. Do not forget to share your mo

In [ ]:
train_result

TrainOutput(global_step=9036, training_loss=0.4689284651719558, metrics={'train_runtime': 8070.1812, 'train_samples_per_second': 8.956, 'train_steps_per_second': 1.12, 'total_flos': 5.221189959863501e+16, 'train_loss': 0.4689284651719558, 'epoch': 3.0})

In [ ]:
# test_results = trainer.predict(
#       test_dataset,
#       metric_key_prefix="test",
#       max_length=max_target_length,
#       num_beams=6
# )

In [ ]:
# test_results

In [ ]:
src_text = [""" I love the food at this place"""]
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

[' I love this food']


In [ ]:
test_results = trainer.predict(
      test_dataset,
      metric_key_prefix="test",
      max_length=max_target_length,
      num_beams=6
)